## 检索器：调优 - 上下文压缩
****
- LLMChainExtractor
- LLMChainFilter
- 多个压缩器组合管道

In [2]:
from langchain_deepseek import ChatDeepSeek
import os

llm = ChatDeepSeek(
    model="Pro/deepseek-ai/DeepSeek-V3",
    temperature=0,
    api_key=os.environ.get("DEEPSEEK_API_KEY"),
    api_base=os.environ.get("DEEPSEEK_API_BASE"),
)

In [3]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

#### 上下文压缩
****


未优化前使用向量库自带能力

In [4]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

documents = TextLoader("test.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
retriever = FAISS.from_documents(texts, OpenAIEmbeddings()).as_retriever()

docs = retriever.invoke("如何进行模型部署")
pretty_print_docs(docs)

Document 1:

3.1 创建知识库
将需要 AI 分析处理的文档上传至知识库中。为确保 DeepSeek 模型能够准确理解文档内容，建议使用"父子分段"模式进行文本处理 - 这种模式能够更好地保留文档的层级结构和上下文关系。如需了解详细的配置步骤，请参考：创建知识库。


3.2 将知识库集成至 AI 应用
在 AI 应用的"上下文"内添加知识库，在对话框内输入相关问题。LLM 将首先从知识库内获取与问题相关上下文，在此基础上进行总结并给出更高质量的回答。


4. 分享 AI 应用
构建完成后，你可以将该 AI 应用分享给他人使用或集成至其它网站内。


阅读更多
除了构建简单的 AI 应用外，你还可以创建 Chatflow / Workflow 搭建更多复杂功能的应用（例如具备文件识别、图像识别、语音识别等能力）。详细说明请参考以下文档：
----------------------------------------------------------------------------------------------------
Document 2:

Deepseek R1 是⽀持复杂推理、多模态处理、技术⽂档⽣成的⾼性能通⽤⼤语⾔模型。本⼿册
为技术团队提供完整的本地部署指南，涵盖硬件配置、国产芯⽚适配、量化⽅案、云端替代⽅
案及完整671B MoE模型的Ollama部署⽅法。模型 参数规
模
计算精
度
最低显存需
求 最低算⼒需求
DeepSeek-R1 (671B) 671B FP8 ≥890GB
2*XE9680（16*H20
GPU）
DeepSeek-R1-Distill-
70B
70B BF16 ≥180GB 4*L20 或 2*H20 GPU
三、国产芯⽚与硬件适配⽅案
1. 国内⽣态合作伙伴动态
企业 适配内容 性能对标（vs
NVIDIA）
华为昇
腾
昇腾910B原⽣⽀持R1全系列，提供端到端推理优化
⽅案 等效A100（FP16）
沐曦
GPU
MXN系列⽀持70B模型BF16推理，显存利⽤率提升
30% 等效RTX 3090
海光
DCU 适配V3/R1模型，性能对标NVIDIA A100 等效A100（BF16）
2. 国产硬件推荐配置
模型参数 推荐⽅案 适⽤场景
1.5B 太初T100加速卡 个⼈开发者原

使用LLMChainExtractor

基础检索器ContextualCompressionRetriever以及LLMChainExtractor，它将迭代最初返回的文档，并从每个文档中仅提取与查询相关的内容

In [6]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "如何进行部署"
)
pretty_print_docs(compressed_docs)

Document 1:

针对金融、法律等高合规需求场景，Dify 提供 本地私有化部署 DeepSeek + Dify，支持 DeepSeek 模型与 Dify 平台同步部署至内网

若提示链接无法访问，你也可以考虑在本地部署 DeepSeek 模型。详细说明请参考 本地部署指南


LLMChainFilter：使用 LLM 链来决定过滤掉哪些最初检索到的文档以及返回哪些文档，而无需操作文档内容

In [7]:
from langchain.retrievers.document_compressors import LLMChainFilter

_filter = LLMChainFilter.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "如何进行部署"
)
pretty_print_docs(compressed_docs)

Document 1:

3.1 创建知识库
将需要 AI 分析处理的文档上传至知识库中。为确保 DeepSeek 模型能够准确理解文档内容，建议使用"父子分段"模式进行文本处理 - 这种模式能够更好地保留文档的层级结构和上下文关系。如需了解详细的配置步骤，请参考：创建知识库。


3.2 将知识库集成至 AI 应用
在 AI 应用的"上下文"内添加知识库，在对话框内输入相关问题。LLM 将首先从知识库内获取与问题相关上下文，在此基础上进行总结并给出更高质量的回答。


4. 分享 AI 应用
构建完成后，你可以将该 AI 应用分享给他人使用或集成至其它网站内。


阅读更多
除了构建简单的 AI 应用外，你还可以创建 Chatflow / Workflow 搭建更多复杂功能的应用（例如具备文件识别、图像识别、语音识别等能力）。详细说明请参考以下文档：
----------------------------------------------------------------------------------------------------
Document 2:

Deepseek R1 是⽀持复杂推理、多模态处理、技术⽂档⽣成的⾼性能通⽤⼤语⾔模型。本⼿册
为技术团队提供完整的本地部署指南，涵盖硬件配置、国产芯⽚适配、量化⽅案、云端替代⽅
案及完整671B MoE模型的Ollama部署⽅法。模型 参数规
模
计算精
度
最低显存需
求 最低算⼒需求
DeepSeek-R1 (671B) 671B FP8 ≥890GB
2*XE9680（16*H20
GPU）
DeepSeek-R1-Distill-
70B
70B BF16 ≥180GB 4*L20 或 2*H20 GPU
三、国产芯⽚与硬件适配⽅案
1. 国内⽣态合作伙伴动态
企业 适配内容 性能对标（vs
NVIDIA）
华为昇
腾
昇腾910B原⽣⽀持R1全系列，提供端到端推理优化
⽅案 等效A100（FP16）
沐曦
GPU
MXN系列⽀持70B模型BF16推理，显存利⽤率提升
30% 等效RTX 3090
海光
DCU 适配V3/R1模型，性能对标NVIDIA A100 等效A100（BF16）
2. 国产硬件推荐配置
模型参数 推荐⽅案 适⽤场景
1.5B 太初T100加速卡 个⼈开发者原

#### 多个压缩器组合管道
***
- 使用DocumentCompressorPipeline轻松地按顺序组合多个压缩器
- 将文档拆解成更小的碎片
- 删除冗余文档
- 串联多个压缩器

In [9]:
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain_text_splitters import CharacterTextSplitter
from langchain.retrievers.document_compressors import EmbeddingsFilter

splitter = CharacterTextSplitter(chunk_size=400, chunk_overlap=0, separator=". ")
redundant_filter = EmbeddingsRedundantFilter(embeddings=OpenAIEmbeddings())
relevant_filter = EmbeddingsFilter(embeddings=OpenAIEmbeddings(), similarity_threshold=0.76)
pipeline_compressor = DocumentCompressorPipeline(
    transformers=[splitter, redundant_filter, relevant_filter]
)

In [10]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline_compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "如何进行部署"
)
pretty_print_docs(compressed_docs)

Document 1:

3.1 创建知识库
将需要 AI 分析处理的文档上传至知识库中。为确保 DeepSeek 模型能够准确理解文档内容，建议使用"父子分段"模式进行文本处理 - 这种模式能够更好地保留文档的层级结构和上下文关系。如需了解详细的配置步骤，请参考：创建知识库。


3.2 将知识库集成至 AI 应用
在 AI 应用的"上下文"内添加知识库，在对话框内输入相关问题。LLM 将首先从知识库内获取与问题相关上下文，在此基础上进行总结并给出更高质量的回答。


4. 分享 AI 应用
构建完成后，你可以将该 AI 应用分享给他人使用或集成至其它网站内。


阅读更多
除了构建简单的 AI 应用外，你还可以创建 Chatflow / Workflow 搭建更多复杂功能的应用（例如具备文件识别、图像识别、语音识别等能力）。详细说明请参考以下文档：
----------------------------------------------------------------------------------------------------
Document 2:

申请 DeepSeek API
访问 DeepSeek API 开放平台，按照页面提示进行申请 API Key。

若提示链接无法访问，你也可以考虑在本地部署 DeepSeek 模型。详细说明请参考 本地部署指南

2. 注册 Dify
Dify 是一个能够帮助你快速搭建生成式 AI 应用的平台，通过接入 DeepSeek API，你可以快速搭建出一个能够易于使用的 DeepSeek AI 应用。

集成步骤
1. 将 DeepSeek 接入至 Dify
访问 Dify 平台，点击右上角头像 → 设置 → 模型供应商，找到 DeepSeek，将上文获取的 API Key 粘贴至其中。点击保存，校验通过后将出现成功提示。


2
----------------------------------------------------------------------------------------------------
Document 3:

国产硬件推荐配置
模型参数 推荐⽅案 适⽤场景
1.5B 太初T100加速卡 个⼈开发者原型验证
14B 昆仑芯K200集群 企业级复杂任务推理